In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [5]:
val model = PipelineModel.load("/home/jovyan/models/spark-ml-model")

model = pipeline_b55dc16529e6


pipeline_b55dc16529e6

In [10]:
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(0))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [15]:
val predictionsDF = model
    .transform(inputDF)
    .withColumn("negative_probability", getProbability($"probability"))
    .select($"tweet", $"negative_probability", $"prediction")

predictionsDF = [tweet: string, negative_probability: double ... 1 more field]


[tweet: string, negative_probability: double ... 1 more field]

In [12]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}.start()

+--------------------+--------------------+----------+
|               tweet|negative_probability|prediction|
+--------------------+--------------------+----------+
|@Juicyanna; Diddy...|  0.8901332679579512|       0.0|
|@sudosushi I know...|  0.7777197158685681|       0.0|
|Blog update #fail...| 0.13637176440714127|       1.0|
|@ohallquist our b...|  0.2979543416869346|       1.0|
|It's back to work...|  0.1468461944857056|       1.0|
|@BillyMcCoy Size ...|  0.4746102174724886|       1.0|
|Do I lose karma p...|  0.5391162718887975|       0.0|
|is wondering what...| 0.20589834531412515|       1.0|
|way too tired but...|  0.9552876900376297|       0.0|
|meh. The 3rd &quo...|  0.8953583465973864|       0.0|
|@MTB_Keith Nice pic | 0.23067385060765142|       1.0|
|OMFG. i didnt exp...|  0.7791850572134945|       0.0|
|Such a good day. ...| 0.16346667183595723|       1.0|
|It's official, I'...|  0.5995152964716891|       0.0|
|Welcome to June, ...| 0.06275747857451555|       1.0|
|Watching 

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@552397f

+--------------------+--------------------+----------+
|               tweet|negative_probability|prediction|
+--------------------+--------------------+----------+
|@DontLetGo14 at h...|   0.639195193416711|       0.0|
|jamEz: Dammit I g...|  0.8096952415109441|       0.0|
|ITS FRIDAY AND IM...|   0.509414330057137|       0.0|
|Wah! can't find m...|  0.8602868675141637|       0.0|
|@PinkM Paka akito...| 0.43680984005490314|       1.0|
|@keremgogus thank...| 0.08933984673423069|       1.0|
|morning all  gett...| 0.21745447986796676|       1.0|
|is being edumacat...|   0.578523779837136|       0.0|
|@craigsbeardwax H...| 0.17051348083951692|       1.0|
|Mon 1st June- Aww...|  0.6697062225155298|       0.0|
|@linds911 Thanks ...|  0.2551624349721602|       1.0|
|going for a morni...| 0.48413181803945005|       1.0|
+--------------------+--------------------+----------+

+--------------------+--------------------+----------+
|               tweet|negative_probability|prediction|
+--------

In [ ]:
val query = predictionsDF
    .withColumn("timestamp", current_timestamp)
    .withWatermark("timestamp", "10 seconds")
    .groupBy(window($"timestamp", "10 seconds", "1 second"), $"prediction")
    .count
    .writeStream
    .outputMode("append")
    .format("console")
    .start()

query.awaitTermination(100000)

-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+-----+
|window|prediction|count|
+------+----------+-----+
+------+----------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+------+----------+-----+
|window|prediction|count|
+------+----------+-----+
+------+----------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+------+----------+-----+
|window|prediction|count|
+------+----------+-----+
+------+----------+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+------+----------+-----+
|window|prediction|count|
+------+----------+-----+
+------+----------+-----+

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+----------+-----+
|              window|prediction|count|
+--------------